In [ ]:
import osmnx as ox
import matplotlib.pyplot as plt
%matplotlib inline
place_name = "balaclava, Victoria, AU" 
graph = ox.graph_from_place(place_name)

In [401]:
import pandas as pd
house_df = pd.read_csv("../data/house_combined.csv")
suburb_list = house_df[['surburb','postcode']].drop_duplicates().reset_index().drop('index', axis = 1)

In [402]:
df2 = house_df[['surburb']].drop_duplicates().reset_index().drop('index', axis = 1)

In [403]:
suburb_postcode_df = house_df[['surburb','postcode']].drop_duplicates().reset_index().drop('index', axis = 1)

In [ ]:
import osmnx as ox
list_1 = {}
for suburb in surburb_list['surburb']:
    if(suburb == 'melbourne'):
        continue
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        buildings = ox.geometries_from_place(place_name, tags = {'amenity': True, 'landuse':['retail','commercial']})
    except Exception:
        list_1[suburb] = None
        continue
    if len(buildings) == 0:
        list_1[suburb] = None
        continue
    for amenity, sub_df in buildings.groupby('amenity'):
        l[amenity] = len(sub_df)
    list_1[suburb] = l    

In [157]:
attribute_list = {**list_1, **list_2}
attribute_df = pd.DataFrame(attribute_list).T
attribute_df.to_csv('../data/raw/attribute_df.csv')
attribute_df = attribute_df[['bench','cafe','childcare','fast_food','kindergarten','parking','restaurant','place_of_worship','school','university']]

In [158]:
attribute_df

,bench,cafe,childcare,fast_food,kindergarten,parking,restaurant,place_of_worship,school,university
south melbourne,11.0,41.0,4.0,17.0,3.0,30.0,40.0,6.0,1.0,NaN
port melbourne,45.0,32.0,1.0,20.0,1.0,118.0,16.0,3.0,3.0,NaN
albert park,139.0,144.0,6.0,59.0,7.0,289.0,172.0,24.0,12.0,2.0
middle park,7.0,5.0,NaN,2.0,2.0,NaN,4.0,1.0,1.0,NaN
east melbourne,102.0,19.0,NaN,1.0,1.0,55.0,8.0,10.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
tallangatta,NaN,3.0,NaN,1.0,NaN,4.0,NaN,1.0,3.0,NaN
eskdale,NaN,1.0,NaN,NaN,NaN,1.0,NaN,2.0,1.0,NaN
dartmouth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
beechworth,1.0,6.0,NaN,NaN,1.0,31.0,4.0,3.0,4.0,NaN


In [ ]:
leisure_list = {}
for suburb in suburb_list['surburb']:
    if(suburb == 'melbourne'):
        continue
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        leisure = ox.geometries_from_place(place_name, tags={'leisure':True})
    except Exception:
        leisure_list[suburb] = None
        continue
    if len(leisure) == 0:
        leisure_list[suburb] = None
        continue
    for leisure, sub_df in leisure.groupby('leisure'):
        l[leisure] = len(sub_df)
    leisure_list[suburb] = l    

In [ ]:
leisure_list = {}
for suburb in suburb_list['surburb']:
    if(suburb == 'melbourne'):
        continue
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        leisure = ox.geometries_from_place(place_name, tags={'leisure':True})
    except Exception:
        leisure_list[suburb] = None
        continue
    if len(leisure) == 0:
        leisure_list[suburb] = None
        continue
    for leisure, sub_df in leisure.groupby('leisure'):
        l[leisure] = len(sub_df)
    leisure_list[suburb] = l    

In [ ]:
leisure_list_2 = {}
for suburb in suburb_list['surburb'][500:]:
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        leisure = ox.geometries_from_place(place_name, tags={'leisure':True})
    except Exception:
        leisure_list_2[suburb] = None
        continue
    if len(leisure) == 0:
        leisure_list_2[suburb] = None
        continue
    for leisure, sub_df in leisure.groupby('leisure'):
        l[leisure] = len(sub_df)
    leisure_list_2[suburb] = l    

In [376]:
leisure_attribute_list = {**leisure_list, **leisure_list_2}
leisure_attribute_list = pd.DataFrame(leisure_attribute_list).T
leisure_attribute_list.to_csv('../data/raw/leisure_df.csv')
leisure_attribute_list = leisure_attribute_list[['garden','park','playground','pitch']]

In [399]:
attribute_conbined = pd.concat([attribute_df,leisure_attribute_list ], axis = 1)

In [400]:
attribute_conbined = attribute_conbined.reset_index().join(surburb_postcode_df).drop('index',axis = 1)
attribute_conbined.to_csv('../data/attribute_conbined.csv')

Download Australia CPI data


In [ ]:
website = 'https://www.rba.gov.au/inflation/measures-cpi.html'
cpi_df = pd.read_html(website)[0]

In [251]:
cpi_df = cpi_df[[(           'Unnamed: 0_level_0',                     'Date'),
            (         'Consumer price index',               'All groups')]]
cpi_df

,Unnamed: 0_level_0,Consumer price index
,Date,All groups
0,2018/2019,2018/2019
1,Sep,1.9
2,Dec,1.8
3,Mar,1.3
4,Jun,1.6
5,2019/2020,2019/2020
6,Sep,1.7
7,Dec,1.8
8,Mar,2.2


In [252]:
filter_cpi_df = pd.DataFrame()
filter_cpi_df['date'] = cpi_df[(           'Unnamed: 0_level_0',                     'Date')]
filter_cpi_df['Consumer price index'] = cpi_df[(         'Consumer price index',               'All groups')]

In [253]:
filter_cpi_df['date'] = cpi_df[(           'Unnamed: 0_level_0',                     'Date')]
filter_cpi_df['Consumer price index'] = cpi_df[(         'Consumer price index',               'All groups')]

In [254]:
filter_cpi_df

,date,Consumer price index
0,2018/2019,2018/2019
1,Sep,1.9
2,Dec,1.8
3,Mar,1.3
4,Jun,1.6
5,2019/2020,2019/2020
6,Sep,1.7
7,Dec,1.8
8,Mar,2.2
9,Jun,−0.3


In [255]:
filter_cpi_df = filter_cpi_df[0:-1]
years = None
data_list = []
for index, item in filter_cpi_df.iterrows():
    mod = index % 5
    if mod == 0:
        years = [int(year) for year in item['date'].split('/') ]
        continue
    cpi = eval(item['Consumer price index'].replace('−','-'))
    if mod == 1:
        data_list.append([years[0],9,cpi])
    elif mod == 2:
        data_list.append([years[0],12,cpi ])
    elif mod == 3:
        data_list.append([years[1],3,cpi ])
    else:
        data_list.append([years[1],6,cpi ])




In [433]:
l = []
for year, month, rate in data_list:
    if month == 9:
        l.append([year, 8, rate])
        l.append([year, month, rate])
        l.append([year, 10, rate])
    elif month == 12:
        l.append([year, 11, rate])
        l.append([year, month, rate])
        l.append([year+1, 1, rate])
    elif month == 3:
        l.append([year, 2, rate])
        l.append([year, month, rate])
        l.append([year, 4, rate])
    else:
        l.append([year, 5, rate])
        l.append([year, month, rate])
        l.append([year, 7, rate])



In [434]:
cpi_df = pd.DataFrame(l, columns=['year','month','Consumer_price_index'])
cpi_df

,year,month,Consumer_price_index
0,2018,8,1.9
1,2018,9,1.9
2,2018,10,1.9
3,2018,11,1.8
4,2018,12,1.8
5,2019,1,1.8
6,2019,2,1.3
7,2019,3,1.3
8,2019,4,1.3
9,2019,5,1.6


In [406]:
website_2 = 'https://www.rba.gov.au/publications/smp/2022/aug/forecasts.html'
cpi_future_df = pd.read_html(website_2)[0]

In [412]:
future_df = pd.DataFrame()
future_df['Consumer_price_index'] = cpi_future_df.iloc[16][1:]
# future_df = future_df.reset_index()

In [413]:
month_list = []
year_list = []
for index, df in future_df.iterrows():
    index = index.split(' ')
    if index[0] == 'Jun':
        month_list.append(6)
    elif index[0] == 'Dec':
        month_list.append(12)
    year_list.append(index[1])

In [414]:
future_df['month'] = month_list
future_df['year'] = year_list

In [421]:
future_df = future_df.reset_index().drop('index', axis = 1)
len(future_df)

6

In [428]:
tmp_list = []
for i in range(len(future_df)):
    if future_df.iloc[i]['month'] == 6:
        tmp_list.append([future_df.iloc[i]['Consumer_price_index'], 6,future_df.iloc[i]['year']])
        tmp_list.append([future_df.iloc[i]['Consumer_price_index'], 7,future_df.iloc[i]['year']])
        tmp_list.append([future_df.iloc[i]['Consumer_price_index'], 8,future_df.iloc[i]['year']])
        tmp_list.append([future_df.iloc[i]['Consumer_price_index'], 9,future_df.iloc[i]['year']])
    else:
        tmp_list.append([future_df.iloc[i]['Consumer_price_index'], 10,future_df.iloc[i]['year']])
        tmp_list.append([future_df.iloc[i]['Consumer_price_index'], 11,future_df.iloc[i]['year']])
        tmp_list.append([future_df.iloc[i]['Consumer_price_index'], 12,future_df.iloc[i]['year']])


In [429]:
tmp_list = pd.DataFrame(tmp_list[1:], columns= future_df.columns)
tmp_list

,Consumer_price_index,month,year
0,6.1,7,2022
1,6.1,8,2022
2,6.1,9,2022
3,7.8,10,2022
4,7.8,11,2022
5,7.8,12,2022
6,6.2,6,2023
7,6.2,7,2023
8,6.2,8,2023
9,6.2,9,2023


In [431]:
cpi_df

,year,month,Consumer price index
0,2018,8,1.9
1,2018,9,1.9
2,2018,10,1.9
3,2018,11,1.8
4,2018,12,1.8
5,2019,1,1.8
6,2019,2,1.3
7,2019,3,1.3
8,2019,4,1.3
9,2019,5,1.6


In [436]:
cpi_combined_df = pd.concat([cpi_df,tmp_list])
cpi_combined_df

,year,month,Consumer_price_index
0,2018,8,1.9
1,2018,9,1.9
2,2018,10,1.9
3,2018,11,1.8
4,2018,12,1.8
...,...,...,...
15,2024,8,3.5
16,2024,9,3.5
17,2024,10,3.0
18,2024,11,3.0
